In [7]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import MobileNetV2
from keras.layers import AveragePooling2D, Dropout, Flatten, Dense, Input, GlobalAveragePooling2D, Conv2D, MaxPooling2D
from keras import regularizers
from keras.models import Model, Sequential
from keras.optimizers import Adam
from keras.applications.mobilenet_v2 import preprocess_input
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import pickle
import os
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import cv2
from keras.models import load_model

In [2]:
def get_pickles():
    combined_dict = pickle.load(open('../Pickles/TTSDict.p', 'rb'))
    
    x_train = combined_dict['train']['images']
    x_test = combined_dict['test']['images']
    
    y_train = combined_dict['train']['labels']
    y_test = combined_dict['test']['labels']
    
    validation = combined_dict['validation']
    
    return x_train, x_test, y_train, y_test, validation 

x_train, x_test, y_train, y_test, validation  = get_pickles() 

In [3]:
print(x_train.shape, x_test.shape)
print(y_train.shape, y_test.shape)

(10000, 250, 250, 3) (992, 250, 250, 3)
(10000, 2) (992, 2)


In [4]:
def get_conv_model(dim):
    '''This function will create and compile a CNN given the input dimension'''
    inp_shape = dim
    act = 'relu'
    drop = .25
    kernal_reg = regularizers.l1(.001)
    optimizer = Adam(lr = .0001)
    
    model = Sequential() 
    
    model.add(Conv2D(64, kernel_size=(3,3),activation=act, input_shape = inp_shape, 
                     kernel_regularizer = kernal_reg,
                     kernel_initializer = 'he_uniform',  padding = 'same', name = 'Input_Layer'))
    model.add(MaxPooling2D(pool_size=(2, 2),  strides = (3,3)))
    
    
    model.add(Conv2D(64, (3, 3), activation=act, kernel_regularizer = kernal_reg, 
                     kernel_initializer = 'he_uniform',padding = 'same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides = (3,3)))
    

    
    model.add(Conv2D(128, (3, 3), activation=act, kernel_regularizer = kernal_reg, 
                     kernel_initializer = 'he_uniform',padding = 'same'))
    model.add(Conv2D(128, (3, 3), activation=act, kernel_regularizer = kernal_reg, 
                     kernel_initializer = 'he_uniform',padding = 'same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides = (3,3)))

    model.add(Flatten())

    
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))

    model.add(Dropout(drop))

    model.add(Dense(2, activation='sigmoid', name = 'Output_Layer'))
    
    model.compile(loss = 'binary_crossentropy', optimizer = optimizer, metrics = ['accuracy'])
    return model


In [10]:
early_stopping = EarlyStopping(monitor='val_loss', verbose = 1, patience=10, min_delta = .00075)
model_checkpoint = ModelCheckpoint('ModelWeights/Normal1.h5', verbose = 1, save_best_only=True,
                                  monitor = 'val_loss')
lr_plat = ReduceLROnPlateau(patience = 5, mode = 'min')
epochs = 2000
batch_size = 100
    

dim = (x_train.shape[1], x_train.shape[2], x_train.shape[3])
normal_model = get_conv_model(dim =dim)
normal_model.load_weights('ModelWeights/Normal1.h5') 

augmentation =ImageDataGenerator(rotation_range = 20, width_shift_range = .2, height_shift_range = .2, 
                                                       horizontal_flip = True, shear_range = .15, 
                                 fill_mode = 'nearest', zoom_range = .15)
augmentation.fit(x_train)
normal_history = normal_model.fit_generator(augmentation.flow(x_train, y_train, batch_size = batch_size),
            epochs = epochs, 
     callbacks = [early_stopping, model_checkpoint, lr_plat], validation_data = (x_test, y_test), verbose= 1)

Epoch 1/2000


INFO:plaidml:Analyzing Ops: 91 of 395 operations complete


100/100 [==============================] - 123s 1s/step - loss: 0.3490 - acc: 0.9719 - val_loss: 0.3235 - val_acc: 0.9814

Epoch 00001: val_loss improved from inf to 0.32348, saving model to ModelWeights/Normal1.h5
Epoch 2/2000
100/100 [==============================] - 110s 1s/step - loss: 0.3502 - acc: 0.9705 - val_loss: 0.3277 - val_acc: 0.9758

Epoch 00002: val_loss did not improve from 0.32348
Epoch 3/2000
100/100 [==============================] - 110s 1s/step - loss: 0.3413 - acc: 0.9735 - val_loss: 0.3117 - val_acc: 0.9798

Epoch 00003: val_loss improved from 0.32348 to 0.31169, saving model to ModelWeights/Normal1.h5
Epoch 4/2000
100/100 [==============================] - 111s 1s/step - loss: 0.3354 - acc: 0.9742 - val_loss: 0.3191 - val_acc: 0.9788

Epoch 00004: val_loss did not improve from 0.31169
Epoch 5/2000
100/100 [==============================] - 115s 1s/step - loss: 0.3348 - acc: 0.9730 - val_loss: 0.3022 - val_acc: 0.9839

Epoch 00005: val_loss improved from 0.31169

In [11]:
import time

time.sleep(1200)

### Mobilenet

In [12]:
def get_mobilenet(dim):
    model = Sequential()
    optimizer = Adam(lr = .0005)
    baseModel = MobileNetV2(weights="imagenet", include_top=False,
        input_tensor=Input(shape=dim))
    
    model.add(baseModel)
    model.add(AveragePooling2D(pool_size=(7, 7)))
    model.add(Flatten(name="flatten"))
    model.add(Dense(256, activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(2, activation="sigmoid", name = 'Output'))

    
    for layer in baseModel.layers:
        layer.trainable = False
        
    model.compile(loss = 'binary_crossentropy', optimizer = optimizer, metrics = ['accuracy'])
    return model

In [13]:
early_stopping = EarlyStopping(monitor='val_loss', verbose = 1, patience=10, min_delta = .00075)
model_checkpoint = ModelCheckpoint('ModelWeights/Mobilenet1.h5', verbose = 1, save_best_only=True,
                                  monitor = 'val_loss')
lr_plat = ReduceLROnPlateau(patience = 5, mode = 'min')
epochs = 2000
batch_size = 32
    

dim = (x_train.shape[1], x_train.shape[2], x_train.shape[3])
mobilenet = get_mobilenet(dim =dim)

    
augmentation =ImageDataGenerator(rotation_range = 20, width_shift_range = .2, height_shift_range = .2, 
                                                       horizontal_flip = True, shear_range = .15, 
                                 fill_mode = 'nearest', zoom_range = .15)
augmentation.fit(x_train)
mobilenet_history = mobilenet.fit_generator(augmentation.flow(x_train, y_train, batch_size = batch_size),
            epochs = epochs, 
     callbacks = [early_stopping, model_checkpoint, lr_plat], validation_data = (x_test, y_test), verbose= 1)

F:\ProgramFiles\conda\envs\FaceMaskEmotionDetection\lib\site-packages\keras_applications\mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


Epoch 1/2000


INFO:plaidml:Analyzing Ops: 1808 of 2052 operations complete


313/313 [==============================] - 152s 485ms/step - loss: 0.1475 - acc: 0.9404 - val_loss: 0.1884 - val_acc: 0.9153

Epoch 00001: val_loss improved from inf to 0.18840, saving model to ModelWeights/Mobilenet1.h5
Epoch 2/2000
313/313 [==============================] - 112s 356ms/step - loss: 0.0911 - acc: 0.9667 - val_loss: 0.1224 - val_acc: 0.9446

Epoch 00002: val_loss improved from 0.18840 to 0.12244, saving model to ModelWeights/Mobilenet1.h5
Epoch 3/2000
313/313 [==============================] - 113s 361ms/step - loss: 0.0757 - acc: 0.9719 - val_loss: 0.1347 - val_acc: 0.9380

Epoch 00003: val_loss did not improve from 0.12244
Epoch 4/2000
313/313 [==============================] - 113s 361ms/step - loss: 0.0882 - acc: 0.9674 - val_loss: 0.0512 - val_acc: 0.9834

Epoch 00004: val_loss improved from 0.12244 to 0.05116, saving model to ModelWeights/Mobilenet1.h5
Epoch 5/2000
313/313 [==============================] - 116s 369ms/step - loss: 0.0614 - acc: 0.9783 - val_loss: 

In [14]:

%%javascript
IPython.notebook.save_notebook()

<IPython.core.display.Javascript object>

In [15]:
os.system(r'%windir%\system32\rundll32.exe powrprof.dll,SetSuspendState Hibernate')

0